Pacotes


In [1]:
import scripts_saneamento as ss # importa fórmulas de tratamento da base
import pandas as pd             # pacote para manipulação de dados em tabela

Importação dos dados

In [2]:
base = pd.read_hdf('base_equip_copia_teste.h5') #importa base

In [15]:
base.shape  # proporção da base (linha, coluna)

(65271, 141)

In [25]:
base_100 = base.iloc[:100] # amostra com 100 linhas para acelerar testes

In [28]:
ss.lista_colunas(base_100).head(10) #lista as colunas presentes na base

,Nome
0,N.º Item - AT
1,Agreg. - AT
2,1 - UNIDADE REGIONAL
3,2 - MUNICÍPIO
4,3 - MUNICÍPIO IBGE
5,4 - PREFIXO DO MUNICÍPIO
6,5 - LOCALIDADE
7,6 - PREFIXO DA LOCALIDADE
8,7 - IMOBILIZADO / INTANGÍVEL
9,10 - ETAPA DO PROCESSO


In [29]:
lista_descricao = base_100.filter(['12 - DESCRIÇÃO DO BEM']) #armazena a coluna de descrição em nova variável

In [32]:
lista_descricao.head(10)

,12 - DESCRIÇÃO DO BEM
12747,BOMBA DOSADORA MARCA PROMINENT MOD DIAFRAGMA N...
12752,"BOMBA SUBMERSIVEL FLYGT 3127.180 CURVA 483 7,5..."
12753,BOMBA SUBMERSIVEL FLYGT 3127.180 HT 483 NS.982...
12754,BOMBA SUBMERSIVEL FLYGT 3127.180 MT 432 NS.972...
12755,BOMBA SUBMERSIVEL FLYGT 3127.180 SERIE 0070145...
12756,Campo vazio
12757,TRANSFORMADOR 150 KVA - ALMIRANTE TAMANDARE
12758,BOMBA SUBMERSIVEL FLYGT 3127.180HT483 NS.97303...
12759,NOBREAK MARCA NHS - COR PRETA MOD PREMIUM PDV ...
12760,MOTOR


In [66]:
indices = lista_descricao.index[lista_descricao['12 - DESCRIÇÃO DO BEM'] == 'Campo vazio']#captura somente o indice e junta numa lista
#indices.append(lista_descricao.index[lista_descricao['12 - DESCRIÇÃO DO BEM'] == 'Campo vazio']) #captura somente o indice e junta numa lista

In [69]:
indices.size

29

In [55]:
lista_descricao['12 - DESCRIÇÃO DO BEM'][12803]

'MOTOR'

Laço que cria nova base com índice descrição e tokens que posteriormente serão utilizados no matching

In [94]:
lista_geral = pd.DataFrame(columns=['Index','Descrição','Tokens']) # cria um DF com as 3 colunas index, descrição e tokens
lista_base = lista_descricao['12 - DESCRIÇÃO DO BEM'].to_list()    # transforma o DF de descrição em uma lista 
for i in lista_base:                                               # itera a lista de descrição linha por linha
    #indice
    indexes = lista_descricao.index[lista_descricao['12 - DESCRIÇÃO DO BEM'] == i] #captura indice
    indexes = indexes.tolist()  #transforma a lista de indices de objeto np para listsa
    
    for index in indexes:       # laço que impede duplicação de indices na base para descrições iguais
        descricao = lista_descricao['12 - DESCRIÇÃO DO BEM'][index]     # captura a descrição pelo index
        tokens = str(descricao).split(' ')                              # Força string e subdivide
        novo_item = {'Index': index,'Descrição': descricao,'Tokens': tokens} # cria nova linha
        lista_geral = lista_geral.append(novo_item, ignore_index=True)  # adciona a nova base

lista_exclusao = ['DE', 'DA', 'DO', 'EM', 'PARA', 'NOS', 'NAS', 'E', ' ']
lista_geral['Tokens'] = lista_geral['Tokens'].replace(lista_exclusao, '') 

In [105]:
lista_geral.head()

,Index,Descrição,Tokens
0,12747,BOMBA DOSADORA MARCA PROMINENT MOD DIAFRAGMA N...,"[BOMBA, DOSADORA, MARCA, PROMINENT, MOD, DIAFR..."
1,12752,"BOMBA SUBMERSIVEL FLYGT 3127.180 CURVA 483 7,5...","[BOMBA, SUBMERSIVEL, FLYGT, 3127.180, CURVA, 4..."
2,12753,BOMBA SUBMERSIVEL FLYGT 3127.180 HT 483 NS.982...,"[BOMBA, SUBMERSIVEL, FLYGT, 3127.180, HT, 483,..."
3,12754,BOMBA SUBMERSIVEL FLYGT 3127.180 MT 432 NS.972...,"[BOMBA, SUBMERSIVEL, FLYGT, 3127.180, MT, 432,..."
4,12755,BOMBA SUBMERSIVEL FLYGT 3127.180 SERIE 0070145...,"[BOMBA, SUBMERSIVEL, FLYGT, 3127.180, SERIE, 0..."


In [50]:
lista_exclusao = ['DE', 'DA', 'DO', 'EM', 'PARA', 'NOS', 'NAS', 'E', '']
lista_token = []
for frase in lista_geral:
    #Elimina numerais do início da frase
    if frase[0].isnumeric():
        aux = frase[1::]
    else:
        aux = frase
    #Analisa quanto à lista de exclusao e elementos unitários
    aux2 = [x for x in aux if x not in lista_exclusao and len(x) > 1]
    lista_token.append(aux2)

#Cria o DataFrame
df_token = pd.DataFrame(lista_token)
#Agrupa e ordena
df_token_cat1 = df_token.groupby([0]).size().reset_index(name='Qtde').sort_values('Qtde', ascending=False).reset_index(drop=True)
df_token_cat1.columns = ['Categoria', 'Qtde']
df_token_cat1['Qtde [%]'] = df_token_cat1.loc[:, 'Qtde']/df_token_cat1.loc[:, 'Qtde'].sum()*100
#print(df_token_cat1.head(30))
return df_token, df_token_cat1

KeyError: 0

In [ ]:
def tokeniza(base_descr):
    #Tokeniza as frases descritivas de cada ativo
    lista_base = base_descr.to_list()
    #Divide em substrings
    lista_geral = []
    for i in lista_base:
        #Força string
        aux = str(i).split(' ')
        lista_geral.append(aux)
    lista_exclusao = ['DE', 'DA', 'DO', 'EM', 'PARA', 'NOS', 'NAS', 'E', '']
    lista_token = []
    for frase in lista_geral:
        #Elimina numerais do início da frase
        if frase[0].isnumeric():
            aux = frase[1::]
        else:
            aux = frase
        #Analisa quanto à lista de exclusao e elementos unitários
        aux2 = [x for x in aux if x not in lista_exclusao and len(x) > 1]
        lista_token.append(aux2)
    
    #Cria o DataFrame
    df_token = pd.DataFrame(lista_token)
    #Agrupa e ordena
    df_token_cat1 = df_token.groupby([0]).size().reset_index(name='Qtde').sort_values('Qtde', ascending=False).reset_index(drop=True)
    df_token_cat1.columns = ['Categoria', 'Qtde']
    df_token_cat1['Qtde [%]'] = df_token_cat1.loc[:, 'Qtde']/df_token_cat1.loc[:, 'Qtde'].sum()*100
    #print(df_token_cat1.head(30))
    return df_token, df_token_cat1

In [ ]:
df_token, df_token_cat1 = ss.tokeniza(lista_descricao)

In [ ]:
df_token.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BOMBA,DOSADORA,MARCA,PROMINENT,MOD,DIAFRAGMA,N.SER,S2CBH16130PVTS120UA,CAP,"130,0",L/H,None,None,None,None,None,None,None,None,None
1,BOMBA,SUBMERSIVEL,FLYGT,3127.180,CURVA,483,"7,5KW",(10CV),NS.0303211,1735RPM,220V/26A,None,None,None,None,None,None,None,None,None
2,BOMBA,SUBMERSIVEL,FLYGT,3127.180,HT,483,"NS.98207117,5KW",(10CV),1735RPM,None,None,None,None,None,None,None,None,None,None,None
3,BOMBA,SUBMERSIVEL,FLYGT,3127.180,MT,432,"NS.97207277,5KW",(10CV),1735RPM,None,None,None,None,None,None,None,None,None,None,None
4,BOMBA,SUBMERSIVEL,FLYGT,3127.180,SERIE,0070145,"7,5KW",1735RPM,220/380V,26/15A,None,None,None,None,None,None,None,None,None,None


In [ ]:
lista_tokens = df_token_cat1.iloc[:,0].unique() #cria array com as palavras únicas

In [ ]:
matches = ss.analisa_match2(df_token_cat1.Categoria,df_token_cat1.Qtde, 69)

In [ ]:
matches

,Ref,Freq_ref,Match,Freq_match,Simple ratio,Partial ratio,Sort ratio,Set ratio,mean ratio,median ratio,desv ratio
0,MOTOR,6,MOTO,2,89,100,89,89,91.75,89.0,4.763
1,MOTO,2,MOTOR,6,89,100,89,89,91.75,89.0,4.763
4,BOMBA,32,MOTOBOMBA,4,71,100,71,71,78.25,71.0,12.557
5,MOTOBOMBA,4,BOMBA,32,71,100,71,71,78.25,71.0,12.557


In [ ]:
matches_100 = matches.loc[:,["Ref", "Freq_ref","Match", "Freq_match", "median ratio", "desv ratio"]]\
     .where(matches.loc[:,"median ratio"] == 100)

In [ ]:
matches_100.to_excel('matches_100.xlsx')

In [ ]:

lista_descricao[(lista_descricao.str.contains('BOMBA', regex=False)) & (lista_descricao.str.contains('DOSADORA', regex=False)) ]

12747    BOMBA DOSADORA MARCA PROMINENT MOD DIAFRAGMA N...
12804    BOMBA DOSADORA PROMINENT SIGMA3 SN.2001069794 ...
12811                                       BOMBA DOSADORA
Name: 12 - DESCRIÇÃO DO BEM, dtype: object

In [ ]:

lista_descricao[lista_descricao.str.contains('DOSADORA', regex=False)]

12747    BOMBA DOSADORA MARCA PROMINENT MOD DIAFRAGMA N...
12804    BOMBA DOSADORA PROMINENT SIGMA3 SN.2001069794 ...
12811                                       BOMBA DOSADORA
Name: 12 - DESCRIÇÃO DO BEM, dtype: object